In [14]:

import tensorflow as tf
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.activations import relu, softmax, sigmoid
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Lambda
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report

**Mounting Google Drive**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Function used**


*  To trim dataset:  https://drive.google.com/file/d/1QADrY4kxUQE5P3Nud32PFe5-a51zTZDu/view?usp=drive_link

compile(data_ptr -> pandas dataframe file, fill_mode -> str)

In [3]:
import pandas as pd
import seaborn as sns
data = pd.read_csv('/content/drive/My Drive/Statistella/train.csv')

In [4]:

data.drop([
"CODE_GENDER",
"DAYS_REGISTRATION",
"FLAG_EMP_PHONE",
"FLAG_MOBIL",
"FLAG_WORK_PHONE",
"HOUR_APPR_PROCESS_START",
"LIVE_CITY_NOT_WORK_CITY",
"LIVE_REGION_NOT_WORK_REGION",
"NAME_TYPE_SUITE",
"OCCUPATION_TYPE",
"ORGANIZATION_TYPE",
"REG_CITY_NOT_LIVE_CITY",
"REG_CITY_NOT_WORK_CITY",
"REG_REGION_NOT_LIVE_REGION",
"REG_REGION_NOT_WORK_REGION",
"REGION_POPULATION_RELATIVE",
"REGION_RATING_CLIENT",
"WEEKDAY_APPR_PROCESS_START",
"ID"], axis =1)

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from pandas.api.types import is_numeric_dtype

encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

for column in data.columns:
    if not is_numeric_dtype(data[column]):
      data[column] = encoder.fit_transform(data[[column]])

import importlib.util, os
spec = importlib.util.spec_from_file_location("fill", "/content/drive/My Drive/Statistella/data_compiler.py")
fill = importlib.util.module_from_spec(spec)
spec.loader.exec_module(fill)

fill.compile(data, "mean");

print(data.head(20))
from sklearn.model_selection import train_test_split
Y = data['TARGET']
X = data.drop(['TARGET', "DAYS_BIRTH"], axis = 1)
X[["DAYS_EMPLOYED", "DAYS_REGISTRATION", "DAYS_ID_PUBLISH", "OWN_CAR_AGE"]] /= 500
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
print(Y_train)

        ID  TARGET  NAME_CONTRACT_TYPE  CODE_GENDER  FLAG_OWN_CAR  \
0   196348       0                 1.0          1.0           1.0   
1   147976       0                 1.0          0.0           1.0   
2    52662       0                 0.0          1.0           1.0   
3   101577       0                 1.0          1.0           1.0   
4   173078       0                 1.0          1.0           0.0   
5    10383       0                 1.0          1.0           1.0   
6    59786       0                 1.0          1.0           1.0   
7   129341       0                 1.0          1.0           1.0   
8   147812       0                 1.0          1.0           1.0   
9    87244       1                 1.0          0.0           0.0   
10   69028       0                 1.0          0.0           0.0   
11   76892       0                 1.0          0.0           0.0   
12  212886       0                 1.0          1.0           1.0   
13   15888       0                

In [ ]:

model1 = Sequential([
    Dense(units = 8, activation = 'relu', bias_regularizer = regularizers.L1(0.01)),
    Dense(units = 4, activation = 'relu'),
    Dense(units = 1, activation = 'sigmoid'),
])
model1.compile(optimizer = Adam(learning_rate = 1e-5), loss = BinaryCrossentropy())
model1.fit(X_train, Y_train, epochs = 30)


In [84]:
from tensorflow.keras.layers import Lambda
model2 = Sequential([
    Dense(units = 2, activation = 'sigmoid', bias_regularizer = regularizers.L2(0.001)),
    Lambda(lambda x:x*10000),
    #Dense(units = 2, activation = 'relu'),
    Dense(units = 2, activation = 'softmax'),
])

model2.compile(optimizer = Adam(learning_rate = 1e-5), loss = SparseCategoricalCrossentropy())
model2.fit(X_train, Y_train, epochs = 4)

Epoch 1/4
5382/5382 [==============================] - 8s 1ms/step - loss: 733.6776
Epoch 2/4
5382/5382 [==============================] - 8s 1ms/step - loss: 659.2159
Epoch 3/4
5382/5382 [==============================] - 8s 1ms/step - loss: 584.0530
Epoch 4/4
5382/5382 [==============================] - 8s 1ms/step - loss: 509.6241


In [7]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(penalty='l2')
model.fit(X_train, Y_train)
y_pred = model.predict(X_test)
print(classification_report(Y_test, y_pred))

ValueError: Solver lbfgs supports only 'l2' or None penalties, got l1 penalty.

In [ ]:
model1.save('/content/drive/My Drive/Statistella/model1.h5')
# Y_pred = model1.predict(X_test)
# submission = []



# # print(Y_pred)

# from sklearn.metrics import roc_curve

# # Compute ROC curve
# fpr, tpr, thresholds = roc_curve(Y_test, Y_pred)

# # Optimal threshold (You can choose based on your criterion)
# optimal_idx = (tpr - fpr).argmax()
# optimal_threshold = thresholds[optimal_idx]
# print("Optimal Threshold:", optimal_threshold)

NameError: name 'model1' is not defined

In [ ]:

model2.save('/content/drive/My Drive/Statistella/model2_B.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [69]:
import h5py

file_path = '/content/drive/My Drive/Statistella/model2_B.h5'



from tensorflow.keras.models import load_model

file_path =  '/content/drive/My Drive/Statistella/model2_A.h5'
model = load_model(file_path)

# Print model summary
model.summary()

predictions = model.predict(X_test)

#f1_score(Y_test, predictions, labels=None, pos_label=1, average = 'binary', sample_weight=None, zero_division='warn')

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_44 (Dense)            (None, 6)                 732       
                                                                 
 lambda_14 (Lambda)          (None, 6)                 0         
                                                                 
 dense_45 (Dense)            (None, 3)                 21        
                                                                 
 dense_46 (Dense)            (None, 2)                 8         
                                                                 
Total params: 761 (2.97 KB)
Trainable params: 761 (2.97 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2440, in predict_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2425, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2413, in run_step  **
        outputs = model.predict_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2381, in predict_step
        return self(x, training=False)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/input_spec.py", line 298, in assert_input_compatibility
        raise ValueError(

    ValueErro

In [86]:
from sklearn.metrics import f1_score
predictions = model2.predict(X_test)
submission1 = []
for i in predictions:
  if i[0] < i[1]:
    submission1.append(0)
  else:
    submission1.append(1)
submissions = np.array(submission1)

submission2 = []
for i in predictions:
  if i[0] > i[1]:
    submission2.append(0)
  else:
    submission2.append(1)
submissions = np.array(submission2)

f1_score(Y_test, submission2, labels=None, pos_label=1, average = 'binary', sample_weight=None, zero_division='warn')

#Best 0.22, 0.15 model2_A submission1
# 0.93 0.12 model2_B submission2


1346/1346 [==============================] - 2s 1ms/step


0.001729106628242075

In [ ]:
# from tensorflow.keras.models import load_model

# # Load the model
# model = load_model('/content/drive/My Drive/Statistella/model2_B.h5')
# model.summary()
!$pip install pytables
import pandas as pd

# Load the .h5 file
df = pd.read_hdf('/content/drive/My Drive/Statistella/model2_B.h5')
print(df.head())

SyntaxError: invalid syntax (<ipython-input-97-34a7eadef088>, line 6)

In [ ]:
drive.flush_and_unmount()